In [ ]:
import pandas as pd
from pymongo import MongoClient

# Conexión a MongoDB
mongo_client = MongoClient("mongodb://localhost:27017/")
db = mongo_client["reddit_db"]
collection = db["posts"]

# Cargar documentos en un DataFrame
cursor = collection.find()
df = pd.DataFrame(list(cursor))

# Convertir fecha si hace falta
df["fecha_utc"] = pd.to_datetime(df["fecha_utc"])


In [ ]:
print(df.head())  # Ver primeras filas
print(df.describe())  # Estadísticas generales
print(df.dtypes)  # Tipos de datos


In [ ]:
#Análisis por variables
#Puntaje promedio
print("Puntaje promedio:", df["score"].mean())
#Promedio de comentarios
print("Comentarios promedio:", df["comentarios"].mean())
#Cantidad de posts por día
df["fecha_dia"] = df["fecha_utc"].dt.date
posts_por_dia = df.groupby("fecha_dia").size()
print(posts_por_dia)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Gráfico de barras: posts por día
posts_por_dia.plot(kind="bar", figsize=(10,5), title="Posts por día")
plt.ylabel("Cantidad de posts")
plt.xlabel("Fecha")
plt.tight_layout()
plt.show()

# Distribución de scores
sns.histplot(df["score"], bins=20, kde=True)
plt.title("Distribución de puntajes")
plt.xlabel("Score")
plt.show()


In [ ]:
import pandas as pd
from pymongo import MongoClient

mongo_client = MongoClient("mongodb://localhost:27017/")
db = mongo_client["reddit_db"]
collection = db["posts"]
df = pd.DataFrame(list(collection.find()))

# Rellenar campos vacíos con strings vacíos
df["texto"] = df["texto"].fillna("")
df["titulo"] = df["titulo"].fillna("")


In [ ]:
import nltk
from nltk.corpus import stopwords
import spacy
import string

nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words("english"))

def limpiar_texto(texto):
    doc = nlp(texto.lower())
    tokens_limpios = [
        token.lemma_ for token in doc 
        if token.is_alpha and token.lemma_ not in stop_words
    ]
    return " ".join(tokens_limpios)

# Aplicar limpieza al campo "titulo"
df["titulo_limpio"] = df["titulo"].apply(limpiar_texto)


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

texto_total = " ".join(df["titulo_limpio"].tolist())
# texto_total = " ".join(df["titulo"].tolist())

wordcloud = WordCloud(width=800, height=400, background_color='white').generate(texto_total)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("🔤 Palabras más comunes en los títulos")
plt.show()


In [ ]:
from textblob import TextBlob

def sentimiento(texto):
    return TextBlob(texto).sentiment.polarity  # De -1 a +1

df["sentimiento"] = df["titulo"].apply(sentimiento)

# Visualizamos
import seaborn as sns

sns.histplot(df["sentimiento"], bins=20, kde=True)
plt.title("Sentimiento de los títulos")
plt.xlabel("Polaridad (-1 = negativo, +1 = positivo)")
plt.show()


In [ ]:
import spacy

# Cargar el modelo
nlp = spacy.load("en_core_web_sm")

def extraer_ubicaciones(texto):
    doc = nlp(texto)
    ubicaciones = [ent.text for ent in doc.ents if ent.label_ == "GPE"]
    return ubicaciones

# Ejemplo de uso
df["ubicaciones"] = df["titulo"].apply(extraer_ubicaciones)

# Ver qué ubicaciones se encontraron
print(df[["titulo", "ubicaciones"]].head())


In [24]:
from geopy.geocoders import Nominatim

# Inicializar el geolocalizador
geolocator = Nominatim(user_agent="geopyExample")

# Función para obtener las coordenadas
def obtener_coordenadas(ubicacion):
    try:
        location = geolocator.geocode(ubicacion)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        return None, None

# Aplicar geolocalización al DataFrame
df["coordenadas"] = df["ubicaciones"].apply(lambda x: obtener_coordenadas(x[0] if x else ""))
df["latitud"] = df["coordenadas"].apply(lambda x: x[0] if x else None)
df["longitud"] = df["coordenadas"].apply(lambda x: x[1] if x else None)

print(df)


                          _id  \
0    680aca68fa65f65856b19cd4   
1    680aca68fa65f65856b19cd5   
2    680aca68fa65f65856b19cd6   
3    680aca68fa65f65856b19cd7   
4    680aca68fa65f65856b19cd8   
..                        ...   
395  680b8aa024f440d1b27eb4b1   
396  680b8aa024f440d1b27eb4b2   
397  680b8aa024f440d1b27eb4b3   
398  680b8aa024f440d1b27eb4b4   
399  680b8aa024f440d1b27eb4b5   

                                                titulo texto  score  \
0    China to Trump: Cancel Tariffs If You are Seri...            1   
1    Dutch town "most likely" trashed 46 artworks b...           15   
2    Argentina's Economy grew more than expected in...           20   
3    Health of Brazil's ex-president Bolsonaro has ...           31   
4    Ahmed al-Sharaa said to seek peace with Israel...           42   
..                                                 ...   ...    ...   
395          German beer sales down at home and abroad           40   
396  Dutch intelligence service AIV

In [25]:
# Contar la cantidad de posts por ubicación
posts_por_ubicacion = df.groupby(["latitud", "longitud"]).size().reset_index(name="cantidad")

# Mostrar los resultados
print(posts_por_ubicacion)


      latitud    longitud  cantidad
0  -37.713748  145.074890         3
1  -34.996496  -64.967282         4
2  -28.816624   24.991639         6
3  -23.328398   32.806606         3
4  -10.333333  -53.200000         7
5   -2.483383  117.890285         4
6    6.311055   20.544753        11
7    9.600036    7.999972         3
8   12.750349  122.731210         3
9   19.097403  -70.302803         3
10  22.351115   78.667743        17
11  23.658512 -102.007710         6
12  25.624262   42.352833         4
13  30.330840   71.247499         1
14  30.812425   34.859476        12
15  32.647531   54.564352        10
16  34.646078  135.515268         6
17  34.814789   38.423178         4
18  35.000066  104.999955        30
19  36.574844  139.239418         2
20  36.638392  127.696119         4
21  36.964335  126.998417         1
22  36.964360  126.998430         7
23  39.294076   35.231663         4
24  39.326068   -4.837979         3
25  39.783730 -100.445882        26
26  43.653482  -79.383935   

In [26]:
import folium
from folium.plugins import MarkerCluster

# Crear un mapa base
m = folium.Map(location=[20,0], zoom_start=2)  # Ubicación central para el mapa mundial

# Crear un MarkerCluster para agrupar marcadores
marker_cluster = MarkerCluster().add_to(m)

# Agregar los marcadores al mapa
for _, row in posts_por_ubicacion.iterrows():
    lat, lon, cantidad = row['latitud'], row['longitud'], row['cantidad']
    if lat and lon:  # Verificar que las coordenadas sean válidas
        folium.CircleMarker(
            location=[lat, lon],
            radius=cantidad * 2,  # Tamaño del marcador proporcional a la cantidad
            popup=f"{cantidad} posts",
            color="blue",
            fill=True,
            fill_color="blue"
        ).add_to(marker_cluster)

# Mostrar el mapa
m.save("mapa_ubicaciones.html")
